#  Dashboard Fires Europe
**Ein Python Projekt auf Basis von Daten der ESA Fire Climate Change Initiative Project Fire**   
*Sophie Horn*

---

## Inhalt

TODO Inhaltsverzeichnis hinzufügen

[Einleitung](#einleitung) <br>
[Download der Dateien](#download-der-dateien-von-der-website)

---

## 1 Einleitung

[Einleitung](#einleitung)| [Download der Dateien](#download-der-dateien-von-der-website) | Spalte 3
-------- | -------- | --------

![Sächsische Schweiz](doc/saechsische_schweiz.jpg)

*Waldbrand Sächsische Schweiz [1]*

In den Sommermonaten dieses Jahres waren hunderte Einsatzkräfte damit beschäftigt Waldbrände in Sachsen und Brandenburg zu bekämpfen. Auch wenn Waldbrände meist durch fahrlässiges Handeln oder Brandstiftung verursacht werden, ist die steigende Waldbrandgefahr eine Folge des Klimawandels [2]. Um die Entwicklung genauer nachvollziehen zu können, soll in dieser Projektarbeit Daten zum Waldbrandgeschehen betrachtet werden. 

Das Open Dataset zu den Waldbränden stammt von Website der „ESA Climate Change Initiative“ [3]. Auf der Seite stehen vorverarbeitet Daten eines "Moderate-resolution Imaging Spectroradiometers" (MODIS) zur Verfügung. 
Die Daten liegen im .nc-Format (Network Common Data Format) vor und enthalten pro Rasterzelle (Auflösung 0,25° x 0,25°) der Aufnahme Informationen z.B. zur insgesamt verbrannten Fläche.  
**TODO Zwei Sätze mehr zum Modis**

Ziel dieser Projektarbeit ist es auf Basis der MODIS-Daten ein Dashboard für Brände in Europa zu erstellen. Dafür werden zuerst alle relevanten Dateien von der Website runtergeladen, um anschließend die Daten zu extrahieren, transformieren und in eine MongoDB-Kollektion hochzuladen. Auf Basis der Datenbank können mit gezielten Abfragen explorative Analysen durchzuführen und das Dashboard erstellt werden. 

---

## 2 Download Data

> In diesem Kapitel ist der automatisierte Download der NetCDF-Datei der „Esa Climate Change Iniative“ mit dem HTML-Pares `beautiful soup` beschrieben. 

Die Dateien der "Esa Climate Change Iniative" stehen auf Website https://data.ceda.ac.uk/neodc/esacci/fire/data/burned_area/MODIS/grid/v5.1/ zur Verfügung. Jede Datei enthält die Daten aus einem Monat. Die Daten aus den verschiedenen Jahren befinden sich auf unterschiedlichen Webseiten. Die Jahreszahl befindet sich in der jeweiligen URL. 

![Dateien](doc/dateien.jpg)

*Download-Bereich ESA Climate Change Initiative*

In [1]:
from bs4 import BeautifulSoup
import requests
from requests import get

In [2]:
import re
def download_files_month(path):
    page = requests.get(path)
    soup = BeautifulSoup(page.text, 'html.parser')
    link_list = list()
    for link in soup.find_all('a'):
        # Download-Links ermitteln
        url = link.get('href')
        if 'nc?download=1' in url:
            if url not in link_list:
                print(url)
                # Datum aus dem Linktitel ermitteln und Filename erstellen 
                datum = re.findall("[0-9]{8}", url)[0] 
                filename = f"data/MODIS_{datum}.nc"
                print(filename)
                link_list.append(url)
                # Datei herunterladen 
                r = requests.get(url, allow_redirects=True)
                open(filename, 'wb').write(r.content)
        else:
            continue

Um die NetCDF-Files herunterzuladen, wird zuerst die jeweilige Webseite mit der Bibliothek `BeautifulSoup` geparst. Anschließend werden die Downloadlinks ermittelt werden. In HTML werden Links mit ein `<a>` gekennzeichnet. Mit `link.get` kann dann die eigentliche Adresse ermittelt werden. Da im geparsten HTML jeder Downloadlink doppelt enthalten ist, wir überprüft ob der Link nicht bereits schon in einer Liste enthalten ist. Im Link Titel ist auch die Datumsinformation enthalten. Diese wird mit Regex extrahiert und als Basis für den Dateienname genutzt. Im letzten Schritt wird dann die eigentliche Datei heruntergeladen. 

Mit einer for-Schleife wird die für die Jahre von 2001-2020 automatisiert durchgeführt. 

In [3]:
if 0: 
    for year in range (2001, 2021):
        path = f"https://data.ceda.ac.uk/neodc/esacci/fire/data/burned_area/MODIS/grid/v5.1/" + str(year)
        print(path)
        download_files_month(path)

---

## 3 Konzept *Data Preperation*

> Im Folgenden wird am Beispiel eines Data Set gezeigt, wie die Variablen im ersten Schritt aus der .nc-Datei mit den Bibliotheken `netCDF` extrahiert werden können. Anschließend werden die Variablen mit `numpy` für die Nutzung in einem `pandas`-Dataframe transformiert. Der Dataframe wird mit weiteren Variablen (z.B. Land mit `reverse_geocoder`) angereichert und auf die notwendige Anzahl an Zeilen reduziert. 

In [70]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from geopy.geocoders import Nominatim
import reverse_geocoder as rg
import time 

import warnings
warnings.filterwarnings('ignore')


### Dateienformat NetCDF

Wie beschrieben liegen die Daten im .nc-Format (Network Common Data Format). Diesen Datenformat wird genutzt, um mehrdimensionale wissenschaftlichen Daten zu speichern. Um die Daten mit Python auszulesen, wird die Bibliothek `netCDF4` verwendet. Die Weiterverarbeitung der Daten erfolgt mit `numpy`-Funktionen. 

In [71]:
fn = 'data/MODIS_20160701.nc'
ds = nc.Dataset(fn)

Als Beispiel-Data Set werden im Folgenden die Daten aus Juli 2016 analysiert. 

In [72]:
list_variables = list()
for var in ds.variables:
    list_variables.append(var)
list_variables

['lat',
 'lat_bnds',
 'lon',
 'lon_bnds',
 'time',
 'time_bnds',
 'vegetation_class',
 'vegetation_class_name',
 'burned_area',
 'standard_error',
 'fraction_of_burnable_area',
 'fraction_of_observed_area',
 'number_of_patches',
 'burned_area_in_vegetation_class']

Insgesamt sind 14 verschiedene Variablen in einem Data Set enthalten. Eine genaue Beschreibung der Variablen stellt die ESA in einer Dokumentation zur Verfügung (Quelle). 
TODO Quelle einfügen

In [73]:
for var in list_variables: 
    array = ds[var][:]
    print(f" Variable {var} mit Shape {array.shape}")

 Variable lat mit Shape (720,)
 Variable lat_bnds mit Shape (720, 2)
 Variable lon mit Shape (1440,)
 Variable lon_bnds mit Shape (1440, 2)
 Variable time mit Shape (1,)
 Variable time_bnds mit Shape (1, 2)
 Variable vegetation_class mit Shape (18,)
 Variable vegetation_class_name mit Shape (18, 150)
 Variable burned_area mit Shape (1, 720, 1440)
 Variable standard_error mit Shape (1, 720, 1440)
 Variable fraction_of_burnable_area mit Shape (1, 720, 1440)
 Variable fraction_of_observed_area mit Shape (1, 720, 1440)
 Variable number_of_patches mit Shape (1, 720, 1440)
 Variable burned_area_in_vegetation_class mit Shape (1, 18, 720, 1440)


Wie bereits erwähnt ermöglicht NetCDF, dass Speichern mehrdimensionaler Variablen. Dabei unterscheidet sich die Anzahl der Dimensionen je Variable. Im nächsten Abschnitt werden die einzeln Variablen transformiert und näher beleuchte. 

### Längen- und Breitengrad

Die zentralen Variablen im Data Set sind das Längen- und Breitenmaß. *Longitude* und *Latitude* geben den Mittelpunkt, der Rasterzelle mit der Auflösung 0,25° x 0,25° an. Die geographische Breite von -90°bis 90° und die geographische Länge von -180° bis 180° sind in zwei verschiedenen Variablen gespeichert. Um die Information nutzen zu können, werden aus den einzelnen Variablen die Geokoordinaten gebildet. 

**(1) Auslesen der Arrays aus dem Data Set**

In [74]:
lat = ds['lat'][:]
lon = ds['lon'][:]
lon

masked_array(data=[-179.875, -179.625, -179.375, ...,  179.375,  179.625,
                    179.875],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

**(2)  Umwandlung der Koordinaten-Vektoren in eine Koordinaten-Matrix**

In [75]:
lons, lats = np.meshgrid(lon, lat)
print(lons)
print(lats)

[[-179.875 -179.625 -179.375 ...  179.375  179.625  179.875]
 [-179.875 -179.625 -179.375 ...  179.375  179.625  179.875]
 [-179.875 -179.625 -179.375 ...  179.375  179.625  179.875]
 ...
 [-179.875 -179.625 -179.375 ...  179.375  179.625  179.875]
 [-179.875 -179.625 -179.375 ...  179.375  179.625  179.875]
 [-179.875 -179.625 -179.375 ...  179.375  179.625  179.875]]
[[ 89.875  89.875  89.875 ...  89.875  89.875  89.875]
 [ 89.625  89.625  89.625 ...  89.625  89.625  89.625]
 [ 89.375  89.375  89.375 ...  89.375  89.375  89.375]
 ...
 [-89.375 -89.375 -89.375 ... -89.375 -89.375 -89.375]
 [-89.625 -89.625 -89.625 ... -89.625 -89.625 -89.625]
 [-89.875 -89.875 -89.875 ... -89.875 -89.875 -89.875]]


Die Funktion `np.meshgrid` interpretiert die Input-Variablen als Matrix-Index. Zum Bespiel erzeugt die Funktion aus zwei Input-Arrays (0,1,3), die Output-Arrays ([0,1,2], [0,1,2,], [0,1,2]) und ([0,0,0], [1,1,1], [2,2,2]). Durch das Glätten der Arrays erhalt man die im Anschluss die Variablen: (0,0), (1,0), (2,0), (0,1, etc. 

**(3) Glätten der Variablen zu 1-dimensionalen Arrays**

In [76]:
lons_flatt = lons.flatten()
lats_flatt = lats.flatten()
lons_flatt

masked_array(data=[-179.875, -179.625, -179.375, ...,  179.375,  179.625,
                    179.875],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [77]:
whole_world = {
    'lon': lons_flatt, 
    'lat': lats_flatt, 
}
df_whole_world = pd.DataFrame(whole_world)
df_whole_world

,lon,lat
0,-179.875,89.875
1,-179.625,89.875
2,-179.375,89.875
3,-179.125,89.875
4,-178.875,89.875
...,...,...
1036795,178.875,-89.875
1036796,179.125,-89.875
1036797,179.375,-89.875
1036798,179.625,-89.875


Die beiden Arrays werden in einen Dataframe übernommen und bilden zusammen den *key* des Dataframe. Die Schritte 1-3 werden für automatisierte Ausführung in die Funktion `create_geoframe` überführt. 

In [78]:
def create_geoframe(dataset):
    lat = dataset['lat'][:]
    lon = dataset['lon'][:]
    lons, lats = np.meshgrid(lon, lat)
    lons_flatt = lons.flatten()
    lats_flatt = lats.flatten()
    whole_world = {
    'lon': lons_flatt, 
    'lat': lats_flatt, }
    geoframe = pd.DataFrame(whole_world)
    return geoframe

### Numerische Varaiblen 

In [79]:
# Variable burned_area mit Shape (1, 720, 1440)
# Variable standard_error mit Shape (1, 720, 1440)
# Variable fraction_of_burnable_area mit Shape (1, 720, 1440)
# Variable fraction_of_observed_area mit Shape (1, 720, 1440)
# Variable number_of_patches mit Shape (1, 720, 1440)

Die Variablen `burned_area`, ` standard_error`, ` fraction_of_burnable_area`, ` fraction_of_observed_area` und ` number_of_patches` haben die gleiche Form und können aus diesem Grund gleich transformiert werden. 

In [80]:
def create_column(dataset,name_variable):
    df_temp = pd.DataFrame()
    # Auslesen des Arrays aus dem DataSet 
    variable = dataset[name_variable][:]
    variable = variable[0]
    # Glätten des Arrays
    variable_flatt = variable.flatten()
    # Erstellen des DataFrames 
    df_temp[name_variable] = variable_flatt
    return df_temp

Ergebnis der Funktion ist ein Dataframe. Da die Variablen immer in Bezug auf die Geokoordinaten in gleicher Reihenfolge angegeben werden, kann der Dataframe mit `concat` an den bestehenden Dataframe gehängt werden. 

In [81]:
df_concept_data_prep = pd.concat([df_whole_world,
            create_column(ds, 'burned_area'),
            create_column(ds, 'standard_error'),
            create_column(ds,'fraction_of_burnable_area'),
            create_column(ds,'fraction_of_observed_area'),
            create_column(ds, 'number_of_patches')], axis = 1, sort = False)

In [82]:
df_concept_data_prep

,lon,lat,burned_area,standard_error,fraction_of_burnable_area,fraction_of_observed_area,number_of_patches
0,-179.875,89.875,0.0,0.0,0.0,0.0,0.0
1,-179.625,89.875,0.0,0.0,0.0,0.0,0.0
2,-179.375,89.875,0.0,0.0,0.0,0.0,0.0
3,-179.125,89.875,0.0,0.0,0.0,0.0,0.0
4,-178.875,89.875,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
1036795,178.875,-89.875,0.0,0.0,0.0,0.0,0.0
1036796,179.125,-89.875,0.0,0.0,0.0,0.0,0.0
1036797,179.375,-89.875,0.0,0.0,0.0,0.0,0.0
1036798,179.625,-89.875,0.0,0.0,0.0,0.0,0.0


### Vegetationsklassen 

In [83]:
#Variable burned_area_in_vegetation_class mit Shape (1, 18, 720, 1440)

burned_area_in_vegetation_class= ds['burned_area_in_vegetation_class'][:]
burned_area_in_vegetation_class = burned_area_in_vegetation_class[0]


Das Data Set enthält neben der Information der insgesamt verbrannte Fläche pro Rasterzelle eine Aufschlüsselung der verbrannten Fläche pro Vegetationsklasse. Die Informationen zur Vegetationsklassen werden im Rahmen des Projekts „Land Cover CCI“ der ESA bestimmt. Im Data Set wird in 18 verschiedene Vegetationsklassen unterschieden:  

In [84]:
vegetation_class = ds['vegetation_class'][:]
vegetation_class

masked_array(data=[ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110,
                   120, 130, 140, 150, 160, 170, 180],
             mask=False,
       fill_value=999999)

In [85]:
vegetation_class_name = ds['vegetation_class_name'][:]
vegetation_class_name[1]

masked_array(data=[b'C', b'r', b'o', b'p', b'l', b'a', b'n', b'd', b',',
                   b' ', b'i', b'r', b'r', b'i', b'g', b'a', b't', b'e',
                   b'd', b' ', b'o', b'r', b' ', b'p', b'o', b's', b't',
                   b'-', b'f', b'l', b'o', b'o', b'd', b'i', b'n', b'g',
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --],
             mask=[False, False, False, False, False, False, False, False,
     

Da die Variable ` vegetation_class_name` keine verständlichen Namen der Vegetationsklassen enthält werden die Namen aus der Dokumentation übernommen. 

In [20]:
name_vegetation_class ={
    10: ['no_data'], 
    20: ['cropland_rainfed'], #Ackerland 
    30: ['cropland_irrigated'], #Ackerland_bewässert
    40: ['50_mosaic_cropland_50_natural_vegetation'], #Ackerland >50%, natürliche Bepflanzung <50%
    50: ['tree_cover_broadleaved_evergreen'], # immergrüne Laubbäume 
    60: ['tree_cover_broadleaved_deciduous'], # laubabwerfend Laubbäume
    70: ['tree_cover_needleleaved_evergreen'], # immmergrüne Nadelbäume
    80: ['tree_cover_needleleaved_deciduous'], # laubabwerfend Laubbäume
    90: ['tree_cover_mixed_leave'], # Mischwald
    100: ['50_mosaic_tree_50_herbaceous'], # Bäume >50%, <50% krautig Bewachsen
    110: ['50_herbaceous_50_tree'], # >50% krautige Bewachsen, <50% Bäu,e 
    120: ['shrubland'], #Buschland
    130: ['grassland'], #Grasland
    140: ['lichens_and_mosses'], #Flechten und Moose
    150: ['sparse_vegetation'], #spärliche Vegetation 
    160: ['tree_cover_flooded_fresh_water'], #geflutete Bäume Süßwasser
    170: ['tree_cover_flooded_saline_water'], # geflutete Bäume Salzwasser
    180: ['shrub_flooded_water'] #geflutetes Buschland
}

Für das Projekt werden die einzelnen Kategorien zusammengefasst: 
* 10: no_data
* 20, 30, 40: cropland
* 100, 110: mosaic_tree_grass
* 120, 130, 140, 150: other_vegetation
* 160, 170, 180: flooded_area

Die Vegetationsklassen 50 -90 werden belassen, da die Untersuchung von Waldbränden im Fokus steht und dabei die genaue Aufsplittung interessant sein kann.

In [86]:
i_vegetation_class = 10 
for burned_area_per_veg_class in burned_area_in_vegetation_class:
    s_vegetation_class= f"{i_vegetation_class}_burned_area"
    burned_area_per_veg_class_flatt= burned_area_per_veg_class.flatten()
    df_concept_data_prep[s_vegetation_class] = burned_area_per_veg_class_flatt
    i_vegetation_class= i_vegetation_class + 10
    #print(a)

In [87]:
df_concept_data_prep

,lon,lat,burned_area,standard_error,fraction_of_burnable_area,fraction_of_observed_area,number_of_patches,10_burned_area,20_burned_area,30_burned_area,...,90_burned_area,100_burned_area,110_burned_area,120_burned_area,130_burned_area,140_burned_area,150_burned_area,160_burned_area,170_burned_area,180_burned_area
0,-179.875,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-179.625,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-179.375,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-179.125,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-178.875,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036795,178.875,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036796,179.125,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036797,179.375,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036798,179.625,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
#Spalte umbennen
df_concept_data_prep['cropland_burned_area'] = df_concept_data_prep['20_burned_area'] + df_concept_data_prep['30_burned_area'] + df_concept_data_prep['40_burned_area']
df_concept_data_prep['mosaic_tree_grass_burned_area'] = df_concept_data_prep['100_burned_area'] + df_concept_data_prep['100_burned_area']
df_concept_data_prep['other_vegetation_burned_area']= df_concept_data_prep['120_burned_area'] + df_concept_data_prep['130_burned_area'] + df_concept_data_prep['140_burned_area']+ df_concept_data_prep['150_burned_area']
df_concept_data_prep['flooded_area_burned_area']= df_concept_data_prep['160_burned_area'] + df_concept_data_prep['170_burned_area'] + df_concept_data_prep['180_burned_area']


In [89]:
df_concept_data_prep.drop(columns=['20_burned_area','30_burned_area','40_burned_area','100_burned_area','110_burned_area','120_burned_area', '130_burned_area', '140_burned_area', '150_burned_area','160_burned_area','170_burned_area','180_burned_area'], inplace=True)

In [90]:
replace_names={ '10_burned_area': name_vegetation_class[10][0] + "_burned_area",
                '50_burned_area': name_vegetation_class[50][0] + "_burned_area",
                '60_burned_area': name_vegetation_class[60][0] + "_burned_area", 
                '70_burned_area': name_vegetation_class[70][0] + "_burned_area", 
                '80_burned_area': name_vegetation_class[80][0] + "_burned_area", 
                '90_burned_area': name_vegetation_class[90][0] + "_burned_area", }
replace_names

{'10_burned_area': 'no_data_burned_area',
 '50_burned_area': 'tree_cover_broadleaved_evergreen_burned_area',
 '60_burned_area': 'tree_cover_broadleaved_deciduous_burned_area',
 '70_burned_area': 'tree_cover_needleleaved_evergreen_burned_area',
 '80_burned_area': 'tree_cover_needleleaved_deciduous_burned_area',
 '90_burned_area': 'tree_cover_mixed_leave_burned_area'}

In [91]:
df_concept_data_prep.rename(columns = replace_names, inplace = True)
df_concept_data_prep

,lon,lat,burned_area,standard_error,fraction_of_burnable_area,fraction_of_observed_area,number_of_patches,no_data_burned_area,tree_cover_broadleaved_evergreen_burned_area,tree_cover_broadleaved_deciduous_burned_area,tree_cover_needleleaved_evergreen_burned_area,tree_cover_needleleaved_deciduous_burned_area,tree_cover_mixed_leave_burned_area,cropland_burned_area,mosaic_tree_grass_burned_area,other_vegetation_burned_area,flooded_area_burned_area
0,-179.875,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-179.625,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-179.375,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-179.125,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-178.875,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036795,178.875,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036796,179.125,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036797,179.375,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036798,179.625,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Die gerade gezeigten Schritte werden ebenfalls als Funktion `create_vegetation_class_breakdown` für die automatisierte Datenverarbeitung bereitgestellt. 

In [92]:
def create_vegetation_class_breakdown(dataset):
    burned_area_in_vegetation_class= dataset['burned_area_in_vegetation_class'][:][0]
    i_vegetation_class = 10 
    df_temp = pd.DataFrame()
    for burned_area_per_veg_class in burned_area_in_vegetation_class:
        s_vegetation_class= f"{i_vegetation_class}_burned_area"
        burned_area_per_veg_class_flatt= burned_area_per_veg_class.flatten()
        df_temp[s_vegetation_class] = burned_area_per_veg_class_flatt
        i_vegetation_class= i_vegetation_class + 10
    df_temp['cropland_burned_area'] = df_temp['20_burned_area'] + df_temp['30_burned_area'] + df_temp['40_burned_area']
    df_temp['mosaic_tree_grass_burned_area'] = df_temp['100_burned_area'] + df_temp['100_burned_area']
    df_temp['other_vegetation_burned_area']= df_temp['120_burned_area'] + df_temp['130_burned_area'] + df_temp['140_burned_area']+ df_temp['150_burned_area']
    df_temp['flooded_area_burned_area']= df_temp['160_burned_area'] + df_temp['170_burned_area'] + df_temp['180_burned_area']
    df_temp.drop(columns=['20_burned_area','30_burned_area','40_burned_area','100_burned_area','110_burned_area','120_burned_area', '130_burned_area', '140_burned_area', '150_burned_area','160_burned_area','170_burned_area','180_burned_area'], inplace=True)
    replace_names={ '10_burned_area': 'no_data' + "_burned_area",
            '50_burned_area': 'tree_cover_broadleaved_evergreen' + "_burned_area",
            '60_burned_area': 'tree_cover_broadleaved_deciduous' + "_burned_area", 
            '70_burned_area': 'tree_cover_needleleaved_evergreen' + "_burned_area", 
            '80_burned_area': 'tree_cover_needleleaved_deciduous' + "_burned_area", 
            '90_burned_area': 'tree_cover_mixed_leave' + "_burned_area", }
    df_temp.rename(columns = replace_names, inplace = True)
    return df_temp

### Transformation des Data Frames 

In diesem Abschnitt werden die numerischen Variablen nähere betrachtet und untersucht ob weitere Transformationsschritte notwendig sind. 

In [95]:
df_concept_data_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036800 entries, 0 to 1036799
Data columns (total 17 columns):
 #   Column                                         Non-Null Count    Dtype  
---  ------                                         --------------    -----  
 0   lon                                            1036800 non-null  float32
 1   lat                                            1036800 non-null  float32
 2   burned_area                                    1036800 non-null  float32
 3   standard_error                                 1036800 non-null  float32
 4   fraction_of_burnable_area                      1036800 non-null  float32
 5   fraction_of_observed_area                      1036800 non-null  float32
 6   number_of_patches                              1036800 non-null  float32
 7   no_data_burned_area                            1036800 non-null  float32
 8   tree_cover_broadleaved_evergreen_burned_area   1036800 non-null  float32
 9   tree_cover_broadleaved_d

In [96]:
for column_name in df_concept_data_prep.columns:
    column = df_concept_data_prep[column_name]
    count = (column == 0).sum()
    print('Anzahl an 0 in Spalte', column_name, ' ist : ', count)

Anzahl an 0 in Spalte lon  ist :  0
Anzahl an 0 in Spalte lat  ist :  0
Anzahl an 0 in Spalte burned_area  ist :  1021636
Anzahl an 0 in Spalte standard_error  ist :  1021636
Anzahl an 0 in Spalte fraction_of_burnable_area  ist :  793999
Anzahl an 0 in Spalte fraction_of_observed_area  ist :  807313
Anzahl an 0 in Spalte number_of_patches  ist :  1021636
Anzahl an 0 in Spalte no_data_burned_area  ist :  1029851
Anzahl an 0 in Spalte tree_cover_broadleaved_evergreen_burned_area  ist :  1033357
Anzahl an 0 in Spalte tree_cover_broadleaved_deciduous_burned_area  ist :  1029262
Anzahl an 0 in Spalte tree_cover_needleleaved_evergreen_burned_area  ist :  1035638
Anzahl an 0 in Spalte tree_cover_needleleaved_deciduous_burned_area  ist :  1035730
Anzahl an 0 in Spalte tree_cover_mixed_leave_burned_area  ist :  1036442
Anzahl an 0 in Spalte cropland_burned_area  ist :  1030119
Anzahl an 0 in Spalte mosaic_tree_grass_burned_area  ist :  1031670
Anzahl an 0 in Spalte other_vegetation_burned_area 

Der gesamte Dataframe enthält keine NULL-Werte, aber eine große Anzahl an 0-Werte. Das ursprüngliche Data Set enthält, wie erwähnt, Informationen zu jeder Rasterzelle. Da jedoch nicht jede Rasterzelle verbrannte Fläche enthält, ist der Wert für die `burned_area` größtenteils 0. Für die weitere Auswertung haben nur Rasterzellen einen Mehrwert, welche verbrannte Fläche enthalten. Damit kann der Dataframe mit `burned_area >0` gefiltert werden und damit die gesamte Größe signifikant reduziert werden. 

Die Variable `fraction_of_observed_area` gibt den Anteils der verbrennbaren Fläche an, die in dem Monat beobachtet werden konnte, z.B. ohne Wolkenbedeckung. In der Dokumentation ist die Empfehlung enthalten nur Rasterzellen in die weitere Analyse einzubeziehen, bei welchen der Anteil größer als 80% ist. Aus diesem Grund wird der Dataframe mit `fraction_of_observed_area  > 0.8` gefiltert. 

In [123]:
df_concept_data_prep = df_concept_data_prep[((df_concept_data_prep['fraction_of_observed_area'] > 0.8) & (df_concept_data_prep['burned_area']>0))]

In [98]:
df_numerical = df_concept_data_prep[['burned_area', 'standard_error','fraction_of_burnable_area','fraction_of_observed_area', 'number_of_patches' ]]

In [99]:
df_numerical.describe()

,burned_area,standard_error,fraction_of_burnable_area,fraction_of_observed_area,number_of_patches
count,15115.0,1.511500e+04,15115.000000,15115.000000,15115.000000
mean,32345114.0,1.145705e+06,0.960430,0.994243,25.589348
std,58782248.0,5.189224e+05,0.118531,0.015193,37.621387
min,53664.0,1.059680e+05,0.006217,0.800460,1.000000
25%,1395281.0,7.267085e+05,0.979965,0.995108,2.000000
50%,6600754.0,9.806570e+05,0.998817,0.999575,8.000000
75%,34560048.0,1.542320e+06,1.000000,1.000000,33.000000
max,696567424.0,2.548772e+06,1.000000,1.000000,289.000000


Die Variable `fraction_of_burnable_area` gibt den Anteil der Fläche pro Rasterzelle an, welche verbrannt werden kann, z.B. keine verbrennbare Fläche sind die Weltmeere. Die Variable `number_of_patches` gibt die Anzahl an verbunden verbrannten Rasterzellen an. Die Variable `standard_error` gibt den Wert der Standardfehler bei der Bewertung der verbrannten Fläche an.  Diese drei Variablen werden bei der Erstellung des Dashboards nicht weiter berücksichtigt, aber verbleiben Variable im Dataframe, da sie weiterführende Betrachtungen notwendig sein können. 

In [100]:
corr_matrix = df_numerical.corr()
corr_matrix

,burned_area,standard_error,fraction_of_burnable_area,fraction_of_observed_area,number_of_patches
burned_area,1.000000,0.566091,0.113234,0.130623,0.693796
standard_error,0.566091,1.000000,0.242964,0.246558,0.730881
fraction_of_burnable_area,0.113234,0.242964,1.000000,0.523380,0.128355
fraction_of_observed_area,0.130623,0.246558,0.523380,1.000000,0.146102
number_of_patches,0.693796,0.730881,0.128355,0.146102,1.000000


Die Korrelation zwischen ` standard_error`, `number_of_patches` und `burend_area` ist hoch. Für das Dashboard wird das nicht weiter betrachtet. Für andere Anwendungsfälle sollte die hohe Korrelation berücksichtig werden.

### Zusätzliche Variablen 

**Variable Land**

Um z.B. Auswerten zu können, welche Länder besonders stark von Bränden betroffen sind, muss das die Landesbezeichnung als zusätzlich Variable ergänzt werden.  Mit *reverse geocoding* wird dabei die Landesbezeichnung aus den Geokoordinaten ermittelt.  

In [124]:
df_concept_data_prep.shape 

(15115, 18)

In [102]:
df_geo = df_concept_data_prep.head(100)

In [35]:
geolocator = Nominatim(user_agent="my_app")
st = time.time()
df_geo['add'] = df_geo.apply(lambda x : (geolocator.reverse((x['lat'], x['lon']), zoom=3, language= 'en')).address, axis = 1)
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 49.73007249832153 seconds


In [36]:
df_geo

,lon,lat,burned_area,standard_error,fraction_of_burnable_area,fraction_of_observed_area,number_of_patches,no_data_burned_area,tree_cover_broadleaved_evergreen_burned_area,tree_cover_broadleaved_deciduous_burned_area,tree_cover_needleleaved_evergreen_burned_area,tree_cover_needleleaved_deciduous_burned_area,tree_cover_mixed_leave_burned_area,cropland_burned_area,mosaic_tree_grass_burned_area,other_vegetation_burned_area,flooded_area_burned_area,add
103243,70.875,72.125,1931928.0,488056.0,0.926680,0.967521,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1931927.0,0.0,Russia
106161,80.375,71.625,53664.0,345052.0,0.920062,0.971039,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53664.0,0.0,Russia
106162,80.625,71.625,53664.0,343751.0,0.902998,0.966064,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53664.0,0.0,Russia
107567,71.875,71.375,912299.0,291183.0,0.585074,0.933804,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,858634.0,53664.0,Russia
109006,71.625,71.125,53664.0,357733.0,0.958995,0.976494,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53664.0,0.0,Russia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133601,100.375,66.875,4561496.0,554328.0,0.996817,0.998758,17.0,0.0,0.0,0.0,0.0,3810191.0,0.0,0.0,965964.0,268323.0,0.0,Russia
133602,100.625,66.875,32037806.0,602866.0,0.979830,0.986818,13.0,0.0,0.0,0.0,0.0,29730226.0,0.0,0.0,1931928.0,1287951.0,0.0,Russia
133690,122.625,66.875,1395281.0,524477.0,0.963755,0.985691,6.0,0.0,0.0,0.0,0.0,1287952.0,0.0,0.0,0.0,107329.0,0.0,Russia
133691,122.875,66.875,1717269.0,449843.0,0.983728,0.987055,1.0,0.0,0.0,0.0,0.0,1609940.0,0.0,0.0,0.0,107329.0,0.0,Russia


Mit der Bibliothek `Nominatim` kann eine große Zahl an Geoinformation abgerufen werden. Das führt aber auch zu einer hohen Durchlaufzeit bei der Ermittlung der Länderbezeichnung. Effektiver ist in diesem Fall die Bibliothek `reverse_geocoder` und aus diesem Grund besser für die großen Datenmengen dieses Anwendungsfalls geeignet. Für die automatisierte Datenverarbeitung wird eine entsprechende Funktion bereitgestellt.  

In [37]:
st = time.time()

coords = tuple(zip(df_geo['lat'], df_geo['lon']))

results_rg = rg.search(coords)
results_admin2 = [x.get('cc') for x in results_rg]

# Optional: insert admin2 results into new df column
df_geo['add_rg'] = results_admin2
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')


Loading formatted geocoded file...
Execution time: 1.8593778610229492 seconds


In [38]:
df_geo

,lon,lat,burned_area,standard_error,fraction_of_burnable_area,fraction_of_observed_area,number_of_patches,no_data_burned_area,tree_cover_broadleaved_evergreen_burned_area,tree_cover_broadleaved_deciduous_burned_area,tree_cover_needleleaved_evergreen_burned_area,tree_cover_needleleaved_deciduous_burned_area,tree_cover_mixed_leave_burned_area,cropland_burned_area,mosaic_tree_grass_burned_area,other_vegetation_burned_area,flooded_area_burned_area,add,add_rg
103243,70.875,72.125,1931928.0,488056.0,0.926680,0.967521,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1931927.0,0.0,Russia,RU
106161,80.375,71.625,53664.0,345052.0,0.920062,0.971039,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53664.0,0.0,Russia,RU
106162,80.625,71.625,53664.0,343751.0,0.902998,0.966064,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53664.0,0.0,Russia,RU
107567,71.875,71.375,912299.0,291183.0,0.585074,0.933804,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,858634.0,53664.0,Russia,RU
109006,71.625,71.125,53664.0,357733.0,0.958995,0.976494,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53664.0,0.0,Russia,RU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133601,100.375,66.875,4561496.0,554328.0,0.996817,0.998758,17.0,0.0,0.0,0.0,0.0,3810191.0,0.0,0.0,965964.0,268323.0,0.0,Russia,RU
133602,100.625,66.875,32037806.0,602866.0,0.979830,0.986818,13.0,0.0,0.0,0.0,0.0,29730226.0,0.0,0.0,1931928.0,1287951.0,0.0,Russia,RU
133690,122.625,66.875,1395281.0,524477.0,0.963755,0.985691,6.0,0.0,0.0,0.0,0.0,1287952.0,0.0,0.0,0.0,107329.0,0.0,Russia,RU
133691,122.875,66.875,1717269.0,449843.0,0.983728,0.987055,1.0,0.0,0.0,0.0,0.0,1609940.0,0.0,0.0,0.0,107329.0,0.0,Russia,RU


In [104]:
def add_country(dataframe):
    coords = tuple(zip(dataframe['lat'], dataframe['lon']))
    results_rg = rg.search(coords)
    results_admin2 = [x.get('cc') for x in results_rg]
    dataframe['country'] = results_admin2
    return dataframe

**Variable Datum**

Zum Vergleichen der Brand-Daten über mehrere Zeiträume fehlt im Data Frame die Datumsinformation. Da für jeden Monat eine einzelne NetCDF-Datei abgelegt wird, kann das Datum aus dem Dateinamen generiert und als zusätzlich Spalte zum Dataframe hinzugefügt werden. 

In [105]:
def add_date(filename, dataframe ):
    string_date = filename[-11:-7] + "-" + filename[-7:-5] + "-" + filename[-5:-3]
    dataframe['date'] = string_date
    return dataframe

In [107]:
df_concept_data_prep = add_date(fn, df_concept_data_prep)

In [108]:
fig = go.Figure(data=go.Scattergeo(
        lon = df_concept_data_prep['lon'],
        lat = df_concept_data_prep['lat'],
        marker = dict(
            symbol = 'square',
            autocolorscale = False,
            line = dict(
            ),
            colorscale = 'Sunset',
            cmin = df_concept_data_prep['burned_area'].min(),
            color = df_concept_data_prep['burned_area'],
            cmax = df_concept_data_prep['burned_area'].max(),
            colorbar_title="Burned Area Total"
        )))
fig.update_geos(projection_type="orthographic")
fig.update_layout(
        title = 'Burned Areas',
    )
fig.show()

TODO Visualisierung erklären und Verbesserung ergänzen

---

## 4 *Data Preperation* und Erstellung einer MongoDB Kollektion

> In diesem Abschnitt werden die vorher Data-Prep-Funktionen auf alle heruntergeladenen Dateien angewendet. Die neu entstanden DataFrames werden in eine `MongoDB` Kollektion hinzugefügt. Dabei wird die Entscheidung für eine Non-SQL-Datenbank erläutert.  

In [109]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import reverse_geocoder as rg
import psycopg2 
from sqlalchemy import create_engine
import json
from pymongo import MongoClient
import warnings
warnings.filterwarnings('ignore')

from func import  *

Der Bespiel Data Frame aus Juli 2016 hat trotz Filterung (nach burend_area und fraction_of_observed_area) noch 15115 Einträge. Für die 20 Jahre zwischen 2001 und 2020 bedeutet diese eine sehr große Datenmenge. Für einzelne Auswertung sind immer nur Teile der Datenmenge insteressant. Deswegen ist es sinnvoll die Daten in eine Datenbank auszulagern und durch gezielte Abfrage Teilmengen nutzbar zu machen. 
In Frage für die bearbeitete Aufgabenstellung kommen sowohl eine relationale, wie ein Non-SQL-Datenbank. Die Data Sets der einzelnen Monate können in eine SQL-Datenbank überführt werden, da sie alle gleich strukturiert sind. Daten sprich, das fehlende relational Schema. Die Daten sind in einer großen Tabelle pro Monat vorhanden. Um die Vorteile eine SQL-Datenbank nutzen zu können, müssen die Daten umstrukturiert werden. Ein Schema könnte so aussehen: 


TODO Schema zeichen 

Für den Anwendungsfall stellt die Änderung des Schemas kein Vorteil dar, da davon auszugehen ist, dass neue Daten ebenfalls als jeweils einzelne Tabelle zur Verfügung gestellt werden. Dies führt zu weiteren Transformationsschritten und Erhöhung der Komplexität. Im Folgenden wird aus diesen Gründen eine MongoDB-Kollektion zur Speicherung der Daten genutzt.

In [110]:
import os
import re
#C:\dev\eda_esa_fire\data
path = "C://dev//eda_esa_fire//data"
dir_list = os.listdir(path)

In Liste `dir_list` sind alle Dateinamen im Ordner "data" mit den vorher runtergeladen Dateien enthalten. 

In [111]:
#!"C:\Program Files\MongoDB\Server\6.0\bin\mongod.exe" --dbpath "C:\mongodb_data"

In [112]:
client = MongoClient()
client = MongoClient('mongodb://localhost:27017/')
mydb = client["esa_fire"]
mycol = mydb["esa_fire_2001"]
print(mydb.name)

esa_fire


In [113]:
print(client.address)
print(client.list_database_names)

('localhost', 27017)
<bound method MongoClient.list_database_names of MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)>


In [125]:
if 0:
        for modis_file in dir_list:
            #print(modis_file)
            #Anwendung der Data Preperation-Schritte
            path  = "data/" + modis_file
            ds = nc.Dataset(path)
            df_data = create_geoframe(ds)
            df_data = add_country(df_data)
            df_data = add_date(path, df_data)

            df_all_data = pd.concat([df_data,
                create_column(ds, 'burned_area'),
                create_column(ds, 'standard_error'),
                create_column(ds,'fraction_of_burnable_area'),
                create_column(ds,'fraction_of_observed_area'),
                create_column(ds, 'number_of_patches'),
                create_vegetation_class_breakdown(ds)], axis = 1, sort = False) 

            df_all_data = df_all_data[((df_all_data['fraction_of_observed_area'] > 0.8) & (df_all_data['burned_area']>0))]
            print(f"Filname: {modis_file} Size: {str(df_all_data.shape[0])}")
            #Hinzufügen der Daten zur Kollektion
            data_dict = df_all_data.to_dict("records")
            mycol.insert_many(data_dict)

In einer `for`-Schleife wird durch die Liste mit Dateinamen iteriert. Für jede Datei werden die Data-Preperation-Schritte durchgeführt und der entstandene Dataframe wird zu der Kollektion hinzugefügt. 

In [115]:
mycol.find().count()

3204039

Die erstellte Kollektion besteht aus 3.2 Millionen Dokumenten. 

Dem Konzept entsprechend kann durch gezielte Anfragen auf die Kollektion Fragestellung direkt beantwortet werden oder Teilmengen für die nähere Analyse extrahiert werden. 

In [118]:
print("Anzahl Einträge mit der Länderinformation Deutschland: ",mycol.count_documents({'country' : 'DE' }))

Anzahl Einträge mit der Länderinformation Deutschland:  96


In [119]:
query={
     '$and': [
            {'country' : 'DE' },
          ]
}
cursor = mycol.find(query)
df =  pd.DataFrame(list(cursor))
df

,_id,lon,lat,country,date,burned_area,standard_error,fraction_of_burnable_area,fraction_of_observed_area,number_of_patches,no_data_burned_area,tree_cover_broadleaved_evergreen_burned_area,tree_cover_broadleaved_deciduous_burned_area,tree_cover_needleleaved_evergreen_burned_area,tree_cover_needleleaved_deciduous_burned_area,tree_cover_mixed_leave_burned_area,cropland_burned_area,mosaic_tree_grass_burned_area,other_vegetation_burned_area,flooded_area_burned_area
0,6306310a28dd00e55a294f9d,11.625,51.875,DE,2002-10-01,160994.0,482436.0,0.925664,0.973029,3.0,160994.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6306310f28dd00e55a299414,11.625,51.875,DE,2002-11-01,697640.0,711347.0,0.925664,0.974973,2.0,697640.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6306310f28dd00e55a299415,11.875,51.875,DE,2002-11-01,53664.0,720374.0,0.934856,0.974486,1.0,53664.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6306310f28dd00e55a29943a,11.625,51.625,DE,2002-11-01,536646.0,818735.0,0.940242,0.979529,1.0,536646.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6306310f28dd00e55a29945b,6.625,51.375,DE,2002-11-01,751305.0,526805.0,0.595106,0.895701,5.0,375652.0,0.0,0.0,160994.0,0.0,53664.0,53664.0,0.0,107329.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,6306348428dd00e55a530aa7,11.875,51.875,DE,2019-07-01,160994.0,560935.0,0.922367,1.000000,3.0,160994.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,6306348428dd00e55a530c02,9.875,50.125,DE,2019-07-01,1073293.0,694338.0,0.971613,1.000000,1.0,375652.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,590311.0,0.0
93,6306348828dd00e55a5347e3,11.875,51.875,DE,2019-08-01,1609940.0,482329.0,0.922367,0.996097,1.0,1609940.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,6306348828dd00e55a534bce,12.875,48.125,DE,2019-08-01,214658.0,767514.0,0.949443,1.000000,3.0,53664.0,0.0,0.0,107329.0,0.0,0.0,53664.0,0.0,0.0,0.0


---

## 5 Abfragen und Visualisierungen 

>In dem Abschnitt "Abfragen und Visualisierungen" werden mit verschiedenen Abfragen und Visualisierungen unterschiedliche Information aus der Datenbank entnommen. Die Abfragen der Datenbank erfolgen über pymongo und werden teilweise mit pandas weiterverarbeitet. Die Visualisierungen werden sowohl mit plotly express, wie auch folium umgesetzt. 

Folgende Fragestellungen werden im Folgenden beantwortet: 
* Query 1: Wie hat sich die Summe der verbrannten Flächen weltweit entwickelt? 
* Query 2: Welche Länder sind besonders stark betroffen? 
* Query 3: Wie verteilt sich die verbrannte Fläche weltweit auf die verschiedenen Vegetationsklassen? 
* Query 4: In welchen Gegenden von Deutschland ist Fläche größer als ein Quadratkilometer verbrannt?

In [128]:
from pymongo import MongoClient
import pandas as pd
import plotly.express as px
import folium
import pandas as pd
import pycountry
import plotly.io as pio

pio.templates.default = "plotly_white"


In [129]:
#!"C:\Program Files\MongoDB\Server\6.0\bin\mongod.exe" --dbpath "C:\mongodb_data"

In [130]:
client = MongoClient()
client = MongoClient('mongodb://localhost:27017/')
mydb = client["esa_fire"]
mycol = mydb["esa_fire_2001"]
print(mydb.name)

esa_fire


### Query 1: Wie hat sich die Summe der verbrannten Flächen weltweit entwickelt? 

In [131]:
list_date = list ()
agg_result= mycol.aggregate(
    [{
    "$group" : 
        {"_id" : "$date", 
         "burned_area" : {"$sum" : 1}
         }
    }])
for i in agg_result:
    list_date.append(i)

In [132]:
df_date = pd.DataFrame(list_date)
df_date = df_date.sort_values(by='_id')

In [133]:
fig = px.bar(df_date, x="_id", y="burned_area", title='Burned Area per Month', color_discrete_sequence=px.colors.qualitative.Prism)
fig.show()

Auf den ersten Blick auf die Grafik fallen die fehlenden Daten in 2019 auf. Die Ursache dafür liegt in der Datenquelle. Aufgrund eines Fehlers sind die Daten auf der Website von Oktober, November und Dezember 2019 in einem separaten Ordner. 

Bei genauerer Betrachtung der Grafik fehlt auf, dass jedes Jahr eine ähnliche Waldbrandentwicklung aufweist. Die meiste verbrannte Fläche findet sich in den Sommermonaten und am niedrigsten sind die Werte im Winter. Ebenfalls lässt sich die Entwicklung der Summe der verbrannten Fläche auf Monatsebene schwierig beurteilen, deswegen ist eine weitere Aggregation auf Jahr sinnvoll.

In [135]:
df_date['year'] = df_date['_id'].str.slice(start=0, stop=4)
df_groupdate = df_date.groupby(by='year').agg(month=('_id', 'first'), sum=('burned_area', 'sum'))
df_groupdate.reset_index(inplace= True)
df_groupdate

,year,month,sum
0,2001,2001-01-01,137834
1,2002,2002-01-01,167281
2,2003,2003-01-01,175374
3,2004,2004-01-01,174907
4,2005,2005-01-01,178833
5,2006,2006-01-01,172256
6,2007,2007-01-01,177836
7,2008,2008-01-01,174292
8,2009,2009-01-01,162955
9,2010,2010-01-01,163529


In [137]:
fig = px.line(df_groupdate, x='month', y='sum',  title='Burned Area per Year', color_discrete_sequence=px.colors.qualitative.Prism)
fig.show()

TODO Urasache erklärenBrandrodung?

### Query 2: Welche Länder sind besonders stark betroffen? 

In [138]:
list_date = list ()
agg_result= mycol.aggregate(
    [{
    "$group" : 
        {"_id" : "$country", 
         "burned_area" : {"$sum" : 1}
         }
    }])
for i in agg_result:
    list_date.append(i)

In [139]:
df_countries = pd.DataFrame(list_date).sort_values(by=['burned_area'], ascending= False)
df_countries = df_countries.head(20)

In [140]:
df_countries['name'] = df_countries['_id'].apply(lambda x: pycountry.countries.get(alpha_2=x).name)

In [142]:
fig = px.bar(df_countries, x="name", y="burned_area",  title='Burned Area per Country', color_discrete_sequence=px.colors.qualitative.Prism)
fig.show()

### Query 3: Wie verteilt sich die verbrannte Fläche weltweit auf die verschiedenen Vegetationsklassen? 

In [143]:

list_veg = list ()
agg_result= mycol.aggregate([{ "$group":
  { "_id" : None,  
    "burned_area"  : { "$sum": "$burned_area" }, 
    "no_data_burned_area"  : { "$sum":"$no_data_burned_area" },
    "tree_cover_broadleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_broadleaved_deciduous_burned_area" },
    "tree_cover_broadleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_broadleaved_deciduous_burned_area" },
    "tree_cover_needleleaved_evergreen_burned_area"  : { "$sum":"$tree_cover_needleleaved_evergreen_burned_area" },
    "tree_cover_needleleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_needleleaved_deciduous_burned_area" },
    "tree_cover_mixed_leave_burned_area"  : { "$sum":"$tree_cover_mixed_leave_burned_area"},
    "cropland_burned_area"  : { "$sum":"$cropland_burned_area" },
    "mosaic_tree_grass_burned_area"  : { "$sum":"$mosaic_tree_grass_burned_area" },
    "flooded_area_burned_area"  : { "$sum":"$flooded_area_burned_area" },
    "other_vegetation_burned_area"  : { "$sum":"$other_vegetation_burned_area" },}
}])
for i in agg_result:
    list_veg.append(i)
list_veg

[{'_id': None,
  'burned_area': 89940704892432.0,
  'no_data_burned_area': 7004942864842.0,
  'tree_cover_broadleaved_deciduous_burned_area': 35183926581228.0,
  'tree_cover_needleleaved_evergreen_burned_area': 807798707721.0,
  'tree_cover_needleleaved_deciduous_burned_area': 988045537799.0,
  'tree_cover_mixed_leave_burned_area': 151480670524.0,
  'cropland_burned_area': 5687008018231.0,
  'mosaic_tree_grass_burned_area': 4188674581006.0,
  'flooded_area_burned_area': 2544568553482.0,
  'other_vegetation_burned_area': 31979288438507.0}]

In [144]:
df_veg = pd.DataFrame(list_veg)

In [145]:
df_veg= df_veg.drop(columns='_id')

In [146]:
import plotly.graph_objects as go

fig1  = go.Figure()
hrz = ["burned_area", "no_data_burned_area","tree_cover_broadleaved_deciduous_burned_area",
       "tree_cover_needleleaved_evergreen_burned_area","tree_cover_needleleaved_deciduous_burned_area", "cropland_burned_area", "mosaic_tree_grass_burned_area", "flooded_area_burned_area", "other_vegetation_burned_area"
       "Final"]
vrt = [ df_veg["burned_area"][0], 
       -df_veg["no_data_burned_area"][0],
       -df_veg["tree_cover_broadleaved_deciduous_burned_area"][0],
       -df_veg["tree_cover_needleleaved_evergreen_burned_area"][0],
       -df_veg["tree_cover_needleleaved_deciduous_burned_area"][0],
       -df_veg["cropland_burned_area"][0],
       -df_veg["mosaic_tree_grass_burned_area"][0],
       -df_veg["flooded_area_burned_area"][0],
       -df_veg["other_vegetation_burned_area"][0],
       None]
fig1.add_trace(go.Waterfall(                 
                  x = hrz, y = vrt,
                  base = 0,
                  measure = [ "absolute","relative",    
                              "relative","relative",
                              "relative","relative" ], 
              decreasing = {"marker":{"color":'rgb(95,70,144)'}},
              totals = {"marker":{"color":'rgb(56,165,166)'}},                    
                ),)                                 
fig1.show()



### Query 4: In welchen Gegenden von Deutschland ist Fläche größer als ein Quadradkilometer verbrannt?

In [147]:
# Querys Mongo DB
query2={
     '$and': [
            { 'country' : 'DE' },
            { 'burned_area'    : { '$gt': 1000000 }}
          ]
}
points =mycol.find(query2)
print("Anzahl der Pixel in Deutschland mit einer verbrannten Fläche größer als 1km2: ",mycol.count_documents(query2))

Anzahl der Pixel in Deutschland mit einer verbrannten Fläche größer als 1km2:  43


In [ ]:
m = folium.Map(location=[51.02700,10.27486],zoom_start=7)
for citie in points:
    folium.Marker( location=[ citie['lat'], citie['lon']], radius=8,tooltip=citie['date'], icon = folium.Icon(color="red", icon='fire')).add_to(m)

Die Punkte mit einer verbrannten Fläche größer als 1km2 werden auf der Karte mit Markern mit "Feuersymbolen" dargestellt. Mit einem Mouse-Over über die Punkte werden die Daten der Brände angezeigt. 

In [ ]:
m

---

## 6 Dashboard Waldbrände in Europa


> In diesem Notebook wird ein Dashboard mit Waldbranddaten aus Europa erstellt. Dafür werden im ersten Schritt die Grafiken für das Dashboard festgelegt und anschließend das Dashboard erzeugt. 
> Was nutze ich dafür? 

In [53]:
from pymongo import MongoClient
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output# Load Data

In [54]:
client = MongoClient()
client = MongoClient('mongodb://localhost:27017/')
mydb = client["esa_fire"]
mycol = mydb["esa_fire_2001"]

In [55]:
df_countries = pd.read_csv('C:\dev\eda_esa_fire\doc\country_codes.csv', on_bad_lines='skip', delimiter=',')
europe = df_countries[df_countries['region'] == 'Europe']['alpha-2'].to_list()

In der CSV-Datei "country_codes.csv" sind Ländernamen inklusive verschiedenen Abkürzungen und Kontinenten enthalten. Mit Hilfe dieser Datei wird eine Liste mit zweistelligen Ländercodes aus Europa erstellt. Später wird der Dataframe ebenfalls genutzt um die vollständigen Namen zu erhalten. 

In [56]:
list_europe = list(mycol.find({"country":{"$in":europe}}))
df_eu = pd.DataFrame(list_europe)
df_eu.head(5)

,_id,lon,lat,country,date,burned_area,standard_error,fraction_of_burnable_area,fraction_of_observed_area,number_of_patches,no_data_burned_area,tree_cover_broadleaved_evergreen_burned_area,tree_cover_broadleaved_deciduous_burned_area,tree_cover_needleleaved_evergreen_burned_area,tree_cover_needleleaved_deciduous_burned_area,tree_cover_mixed_leave_burned_area,cropland_burned_area,mosaic_tree_grass_burned_area,other_vegetation_burned_area,flooded_area_burned_area
0,630630b328dd00e55a25462f,-3.625,56.125,GB,2001-01-01,804970.0,545465.0,0.829803,0.959910,3.0,751305.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53664.0,0.0
1,630630b328dd00e55a254630,-3.625,55.875,GB,2001-01-01,2039257.0,707836.0,0.938498,0.977980,1.0,1502610.0,0.0,0.0,0.0,0.0,0.0,375652.0,0.0,160994.0,0.0
2,630630b328dd00e55a254631,22.875,53.125,PL,2001-01-01,3005221.0,838988.0,0.996411,0.981294,1.0,0.0,0.0,590311.0,0.0,0.0,0.0,0.0,0.0,53664.0,2361245.0
3,630630b328dd00e55a254632,24.625,51.875,UA,2001-01-01,1609940.0,720975.0,0.974907,0.963874,3.0,965964.0,0.0,429317.0,0.0,0.0,0.0,0.0,0.0,214658.0,0.0
4,630630b328dd00e55a254633,26.125,51.875,UA,2001-01-01,21948848.0,929124.0,0.991455,0.826491,2.0,1287952.0,0.0,268323.0,0.0,0.0,0.0,697640.0,0.0,1985592.0,17709340.0


Mit der Query `"country":{"$in":europe}}` wird ein Dataframe mit alle Waldbranddaten aus Europa erzeugt. 

In [57]:
df_eu['year'] = df_eu['date'].str.slice(start=0, stop=4)

Zur besseren Visualisierung wird eine Jahr-Spalte ergänzt. 

### Festlegung der Grafiken 

In [ ]:
# TODO Grafiken beschreiben 

In [58]:
df_test = df_eu[(df_eu['year'] == '2016') & (df_eu['country'] == 'IT')]

**Area Plot: Summe der verbrannten Fläche pro Monat**

In [59]:
def create_area_plot(df):
    df = df.groupby(by= 'date').sum().reset_index()
    fig = px.area(df, x="date", y="burned_area")
    fig.update_traces(line_color='#FD7207')
    fig.update_layout(title = 'Burend Area per Month', plot_bgcolor= "rgb(250, 250, 250)",paper_bgcolor="rgb(255, 255, 255)")
    return fig

In [60]:
fig_1 = create_area_plot(df_test)
fig_1.show()

**Treemap: Summe der verprannten Fläche pro Vegetationsklasse**

In [61]:
def create_tree_map(df):
    df = df.groupby(by= 'year').sum().reset_index()
    df= df.drop(columns= ['burned_area', 'year', 'lon', 'lat', 'standard_error', 'fraction_of_burnable_area', 'fraction_of_observed_area', 'number_of_patches'] )
    df = df.transpose().reset_index()
    df = df[(df[0] != 0)]
    fig = px.treemap(df, 
        path = [px.Constant('All'), 'index'], 
        color = 0, 
        color_continuous_scale='Emrld',
        values = 0)
    fig.update_layout(title = 'Burend Area per Vegetation Class')
    return fig

In [62]:
fig_2 = create_tree_map(df_test)
fig_2.show()

**Geo-Scatter-Plot: Lage der verbrannten Fläche**

In [63]:
def create_scatter_geo(df):
    fig = go.Figure(data=go.Scattergeo(
         lon=df["lon"], lat= df["lat"], 
         mode = 'markers',
         marker = dict(
            size = 6,
            reversescale = True,
            symbol = 'square',
            line = dict(
                width=0.5,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'Hot',
            cmin = 0,
            color = df['burned_area'],
            cmax = df['burned_area'].max(),
            colorbar_title="Burend Area"
        ))
    )
    fig.update_layout(
        title = 'Burned Area',
        geo = dict(
            scope='europe',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
        ),)
    return fig

In [64]:
fig_3 = create_scatter_geo(df_test)
fig_3.show()

### Dashboard

Im Dashboard sollen die oben erstellten Grafiken dargestellt werden. Über zwei Drop-Down-Menüs sollen die Grafiken an Länder und Jahreszahlen angepasst werden. Im ersten Schritt müssen für die Menüs die Werte und Labels gerneriert werden. 

In [65]:
df_labels = df_countries[df_countries['region'] == 'Europe'][['name', 'alpha-2']]
df_labels.rename(columns={'name': 'label', 'alpha-2': 'value'}, inplace = True)
option_countries= df_labels.to_dict('records')

Das Drop-Down-Menü der Länder wird über den oben eingelesen Länder-Dataframe erzeugt, wobei die Labels die vollständigen Länderbezeichnungen sind und die Values, die im Datensatz verwendet Kurzbezeichnungen. 

In [66]:
unique_dates  = df_eu['year'].unique()
option_dates =  list()
for data in unique_dates: 
    date_dict = {'label': data, 'value': data}
    option_dates.append(date_dict)

*Values* und *Lables* des zweiten Drop-Down-Menüs sind die Jahresbezeichnungen aus dem Datensatz. 

In [67]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Wild Fires Europe", style = {
                    'color': '#3C5C79',
                    'font-family': '"Open Sans", verdana, arial, sans-serif'
                }),
    html.Label([
        "Country",
    dcc.Dropdown(id="selected_country",
                 options= option_countries,
                 multi=False,
                 style={'width': "40%"}
                 )],style = {
                    'color': '#3C5C79',
                    'font-family': '"Open Sans", verdana, arial, sans-serif'
                }),
    html.Label([ "Years",
    dcc.Dropdown(id= "selected_year",
                 options= option_dates,
                 multi=False,
                 style={'width': "40%"}
                 ), ], style = {
                    'color': '#3C5C79',
                    'font-family': '"Open Sans", verdana, arial, sans-serif'
                }), 
    html.Div([ 
            html.Div([
                dcc.Graph(id = 'fig_1'),
            ], className='three columns'),
            html.Div([
                dcc.Graph(id = 'fig_2' ),
            ], className='three columns'),
            html.Div([
                dcc.Graph(id = 'fig_3' ),
            ], className='three columns'),
        ], className = 'row'),
    ])


@app.callback(
    [Output('fig_1' , 'figure'),
     Output('fig_2' , 'figure'), 
     Output('fig_3' , 'figure')],
    Input("selected_country", "value"), 
    Input("selected_year", "value")
)

def update_figure(selected_country, selected_year):

    if not selected_country:
        selected_country = 'IT'
        selected_year = '2016'
    dff = df_eu.copy()
    dff = dff[dff["country"] == selected_country]
    dff = dff[dff["year"] == selected_year]

    fig_1 = create_scatter_geo(dff)
    fig_2 = create_area_plot(dff)
    fig_3 = create_tree_map(dff)
    return [fig_1, fig_2, fig_3]
app.run_server(mode='inline')

Verherende Waldbrandjahr von 2010-2020 in Deutschland waren vor allem 2015 und 2018 (Quelle). Filter man nach Deutschland und 2018 ermöglicht das Dashboard einen Überlick über die Situation. Interessant in dem Jahr ist die Aufteilung der Brände nach Vegetationsklassen. Auffällig ist die große Zahl an verbrannter Fläche bei *flooded_area*. Ein Ursache dafür kann der verheerend [Moorbrand in Meppen 2018](https://www.ndr.de/nachrichten/niedersachsen/osnabrueck_emsland/Moorbrand-Landkreis-Emsland-uebt-Kritik-an-Bundeswehr,moorbrand1172.html) gewesen sein, welcher sich im September ereignet hat. Meppen lässt sich als Waldbrandgebiet auf der Karte wiederfinden. Im  Flächendiagram ist ebenfalls im September ein Pike. 

---

## 7 Ausblick und Fazit

---

## Anhang

### Anhang 1- Quellen

### Anhang 2- Design Entscheidungen 